In [1]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from selenium import webdriver
import os

In [2]:
!which chromedriver

/usr/local/bin/chromedriver


In [3]:
#executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
#browser = Browser('chrome', **executable_path, headless=False)

In [4]:
# Scrape the [NASA Mars News Site](https://mars.nasa.gov/news/) and collect the latest News Title and Paragraph Text. 
# Assign the text to variables that you can reference later.

url = 'https://mars.nasa.gov/news/'
# Retrieve page with the requests module
response = requests.get(url)
# Create BeautifulSoup object; parse with 'html.parser'
soup = bs(response.text, 'html.parser')

# Examine the results, then determine element that contains sought info
#print(soup.prettify())


In [5]:
news_title = soup.find('div', class_='content_title').text
print(f"Most Recent Title: {news_title}")
news_p = soup.find('div', class_='article_teaser_body')
print(f"First Paragraph: {news_p}")

# NOTE: I've tried muptiple ways and keep getting a null object returned for article teaser body     

Most Recent Title: 

NASA's Opportunity Rover Mission on Mars Comes to End


First Paragraph: None


In [6]:
# Use splinter to navigate the site and find the image url for the current Featured Mars Image and 
# assign the url string to a variable called `featured_image_url`.

# https://splinter.readthedocs.io/en/latest/drivers/chrome.html
!which chromedriver

/usr/local/bin/chromedriver


In [7]:
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
browser = Browser('chrome', **executable_path, headless=False)

url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)
html = browser.html
soup = bs(html, 'html.parser')

In [8]:
# Navigate to the page with the full, large image
browser.click_link_by_id('full_image')

#browser.click_link_by_partial_text('more info')

In [9]:
browser.click_link_by_partial_text('more info')

In [12]:
# use bs to image source
html = browser.html
soup = bs(html, 'html.parser')
image_url = soup.find('img').get('src')
base_url = "https://www.jpl.nasa.gov/spaceimages"
featured_img_url = base_url + image_url
print (f"The featured image's URL is: {featured_img_url}")

The featured image's URL is: https://www.jpl.nasa.gov/spaceimages/assets/images/logo_nasa_trio_black@2x.png


In [19]:
### Mars Weather

# Visit the Mars Weather twitter account [here](https://twitter.com/marswxreport?lang=en) 
# Scrape the latest Mars weather tweet from the page. 
# Save the tweet text for the weather report as a variable called `mars_weather`.

twitter_wx_url = "https://twitter.com/marswxreport?lang=en"
tweet_response = requests.get(twitter_wx_url)

tweet_soup = bs(tweet_response.text, 'html.parser')
mars_weather = tweet_soup.find('p', class_='TweetTextSize TweetTextSize--normal js-tweet-text tweet-text').text

print("Today's weather on Mars:")
print(mars_weather)

Today's weather on Mars:
InSight sol 92 (2019-03-01) low -94.4ºC (-137.9ºF) high -12.9ºC (8.8ºF)
winds from the SW at 4.6 m/s (10.2 mph) gusting to 10.4 m/s (23.2 mph)
pressure at 7.20 hPapic.twitter.com/zxXhRFOwTo


In [20]:
### Mars Facts

# Visit the Mars Facts webpage [here](http://space-facts.com/mars/) 
# Use Pandas to scrape the table containing facts about the planet including Diameter, Mass, etc.
mars_facts_url = "http://space-facts.com/mars/"
mars_table = pd.read_html(mars_facts_url)
mars_table



[                      0                              1
 0  Equatorial Diameter:                       6,792 km
 1       Polar Diameter:                       6,752 km
 2                 Mass:  6.42 x 10^23 kg (10.7% Earth)
 3                Moons:            2 (Phobos & Deimos)
 4       Orbit Distance:       227,943,824 km (1.52 AU)
 5         Orbit Period:           687 days (1.9 years)
 6  Surface Temperature:                  -153 to 20 °C
 7         First Record:              2nd millennium BC
 8          Recorded By:           Egyptian astronomers]

In [21]:
# Use Pandas to convert the data to a HTML table string.
mars_facts_df = mars_table[0]
mars_facts_df

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.42 x 10^23 kg (10.7% Earth)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.52 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-153 to 20 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [ ]:
### Mars Hemispheres

# Visit the USGS Astrogeology site [here]:
# (https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars) to obtain high resolution images for each of Mar's hemispheres.

# * You will need to click each of the links to the hemispheres in order to find the image url to the full resolution image.

# Save both the image url string for the full resolution hemisphere image, and the Hemisphere title containing the hemisphere name. 
# Use a Python dictionary to store the data using the keys `img_url` and `title`.

#Append the dictionary with the image url string and the hemisphere title to a list. 
# This list will contain one dictionary for each hemisphere.

# Example:
# hemisphere_image_urls = [
#    {"title": "Valles Marineris Hemisphere", "img_url": "..."},
#    {"title": "Cerberus Hemisphere", "img_url": "..."},
#    {"title": "Schiaparelli Hemisphere", "img_url": "..."},
#    {"title": "Syrtis Major Hemisphere", "img_url": "..."},
# ]